### Finding the main method of a java file and adding @Benchmark annotation

In [25]:
import re

# Define the path to the Java file
java_file_path = 'sample.java'

with open(java_file_path, 'r') as file:
    java_code = file.read()

pattern = r'(?<!\/\/)(?<!\/\*)\b(?:public\s+|private\s+|protected\s+|static\s+|final\s+|native\s+|synchronized\s+|abstract\s+|transient\s+)*[\$_\w<>\[\]]*\s+\w+\s*\([^\)]*\)?\s*\{'


# Find all occurrences of method declarations in the Java code
matches = re.finditer(pattern, java_code)

modified_java_code = ''
previous_end_index = 0

# Loop through each method occurrence
for match in matches:
    
    start_index = match.start()
    end_index = match.end()

    method_declaration = java_code[start_index:end_index]

    # Insert @Benchmark annotation just above the method declaration
    modified_java_code += java_code[previous_end_index:start_index] + '@Benchmark\n' + method_declaration

    previous_end_index = end_index

modified_java_code += java_code[previous_end_index:]

# Write the modified Java code back to the file
with open(java_file_path, 'w') as file:
    file.write(modified_java_code)

<re.Match object; span=(26, 66), match='public static void main(String[] args) {'>
<re.Match object; span=(373, 402), match='public int sum(int a, int b){'>
